# Model Ensembles

In this notebook we will implement stacking ensembles models for online shoppers dataset.

# Code and Tools

Our code of dataset preprocessing is saved in the same folder with this report in descion_tree.py. The data set is also at the same location called "online_shoppers_intention.csv". In order to perform stacking. Besides the packages from skearn such as model_selection and accuracy_score, we also used joblib which provides lightweight pipelining in Python. 

In [2]:
from Desicion_Tree import *
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import joblib

The data is preprocessed by calling functions from the external py file before being used for training and testing. x, y are extracted as features and targets.

In [3]:
data_frame_os = read_data_return_frame("online_shoppers_intention.csv")

x, y, class_names, feature_names = preprocess_df(data_frame_os)

After that, a test size of 0.25 and a random state of 21 are set. This is determined by examing different params pairs and comparing the final results.

In [15]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.25, random_state = 21)

# Models

Next, four pretrained models are loaded to persist to disk transparently and to avoid running reduplicated tasks. dtr, nb, nn, knn were generated using joblib.dump function, representing Decision Tree classifier, Categorical NB classifier, MLP classifier and K Neighbors classifier. All these four models are trained models from group members individual session(only 1 out of 3 classifiers of Bayesian was selected as example). 

In [5]:
dtr = joblib.load("model/dtr.joblib")
nb = joblib.load("model/nb.joblib")
nn = joblib.load("model/mlp.joblib")
knn = joblib.load("model/knn.joblib")

# VotingClassifier

We now define the voting mechanism by using the VotingClassifier of scikit-learn.

In [16]:
voting_clf = VotingClassifier(estimators=[('dtr', dtr), ('knn', knn), ('nb', nb), ('nn', nn)], voting='hard')

To use this voting classifier we need to provide an id for each classifier and the classifiers themselves, and a voting parameter which can be set to hard or soft. If this is set to ‘hard’, it uses the predicted class labels for a majority rule voting, if this is set to ‘soft’, it predicts the class label based on the argmax of the sums of the predicted probabilities.

We then fit the voting classifier.(Module CEGE0004 Week-06 Lecture Slides)

In [17]:
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('dtr',
                              DecisionTreeClassifier(min_samples_leaf=5,
                                                     min_samples_split=5)),
                             ('knn',
                              KNeighborsClassifier(metric='cosine',
                                                   n_neighbors=1)),
                             ('nb', CategoricalNB(alpha=100)),
                             ('nn',
                              MLPClassifier(activation='tanh',
                                            hidden_layer_sizes=(50, 50),
                                            max_iter=1000))])

We also need to fit each classifier independently

In [18]:
for clf in (dtr, nb, nn, knn):
    clf.fit(x_train, y_train)

# Accuracy

In [19]:
for clf in (dtr, nb, nn, knn,  voting_clf):
    print(clf.__class__.__name__)
    y_train_pred = clf.predict(x_train)
    print('\ttrain:', accuracy_score(y_train, y_train_pred))
    y_test_pred = clf.predict(x_test)
    print('\ttest:', accuracy_score(y_test, y_test_pred))

DecisionTreeClassifier
	train: 0.9089434411160376
	test: 0.7885176775867662
CategoricalNB
	train: 0.8458959662593274
	test: 0.843658773921505
MLPClassifier
	train: 0.8456796798961825
	test: 0.843658773921505
KNeighborsClassifier
	train: 0.9998918568184276
	test: 0.7606227700291923
VotingClassifier
	train: 0.8460041094408998
	test: 0.8439831333117094


Stacking Classifier of Voting Classifier reports 84.4% on Test data which is higher than every other type of classifiers. CategoricalNB and MLPClassifier have the same test accuracy on this dataset, which is only 0.03% lower than Voting Classifier.

## References

Lipani, A. (2021) UCL (University College London)Module CEGE0004 Week-05 Practical Material  
Joblib: running Python functions as pipeline jobs, joblib. https://joblib.readthedocs.io/en/latest/